# Aim

- filter full synapse datatframe to pre- or postsynpases of the inhibitory and excitatory cells with a skeleton and with mitochondria 

In [1]:
import pandas 

In [2]:
inh_df = pandas.read_csv("smo_data/inh_cell.csv", header=None, names=["cell_id"])
pyr_df = pandas.read_csv("data/pyc_analysis_ids.csv", header=None, names=["cell_id"])

cell_ids = pandas.concat((inh_df, pyr_df))["cell_id"].unique()

In [3]:
syn_df = pandas.read_csv("smo_data/synapse.csv", index_col=0)

In [4]:
is_pre = syn_df["pre_root_id"].isin(cell_ids)
is_post = syn_df["post_root_id"].isin(cell_ids)


In [5]:
len(cell_ids)

384

In [6]:
len(syn_df.loc[is_pre]["pre_root_id"].unique())

372

In [7]:
len(syn_df.loc[is_post]["post_root_id"].unique())

384

In [8]:
import numpy as np

In [9]:
syn_df["cell_id"] = np.nan
syn_df = syn_df.astype({"cell_id": "Int64"})
syn_df["synapse_type"] = np.nan

syn_df.loc[is_pre, "cell_id"] = syn_df.loc[is_pre, "pre_root_id"]
syn_df.loc[is_pre, "synapse_type"] = "pre"

syn_df.loc[is_post, "cell_id"] = syn_df.loc[is_post, "post_root_id"]
syn_df.loc[is_post, "synapse_type"] = "post"



syn_df.loc[syn_df["pre_root_id"]==syn_df["post_root_id"], "synapse_type"]="autapse"

In [10]:
df = syn_df.loc[is_pre | is_post]

In [11]:
len(df["cell_id"].unique())

384

In [12]:
len(df)

929375

In [13]:
# voxel resolution in nm as found in computedensitystats.py

VOXELRES = [3.58, 3.58, 40.0]  
for c, s in zip("xyz", VOXELRES):
    df[c] = df["ctr_pos_{:s}_vx".format(c)]*s


/home/pfeiffer/miniconda3/envs/micronsbinder/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [14]:
df = df[["cell_id", "synapse_type", "x", "y", "z", "pre_root_id", "post_root_id"]]
df = df.reset_index(drop=False)


In [15]:
# duplicate synapses where both pre and post are in the cell ids
hdf = df.loc[(df["synapse_type"]=='post') & (df["pre_root_id"].isin(cell_ids))]
hdf = hdf.reset_index(drop=True)
hdf["cell_id"]=hdf["pre_root_id"]
hdf["synapse_type"]="pre"


In [16]:
df=pandas.concat((df, hdf)).reset_index(drop=True)

In [21]:
df[["x", "y", "z"]]

,x,y,z
0,251301.68,234396.92,40600.0
1,356614.54,186661.20,50760.0
2,334608.28,254180.00,20000.0
3,383718.72,252776.64,8800.0
4,332846.92,174825.72,4480.0
...,...,...,...
935676,338109.52,139254.84,53200.0
935677,339584.48,147037.76,73080.0
935678,337923.36,165306.50,68000.0
935679,336920.96,152329.00,80760.0


In [17]:
df.head(3)

,id,cell_id,synapse_type,x,y,z,pre_root_id,post_root_id
0,2715757,648518346349538285,pre,251301.68,234396.92,40600.0,648518346349538285,648518346349536759
1,12517,648518346349537255,post,356614.54,186661.20,50760.0,648518346342796822,648518346349537255
2,1961413,648518346349537649,post,334608.28,254180.00,20000.0,648518346344080496,648518346349537649


In [18]:
df.to_csv("smo_data/synapse_filtered.csv")

In [19]:
hdf = df.groupby(by=["synapse_type", "cell_id"])[["x"]].count()
hdf = hdf.rename(columns={"x":"count"})

In [20]:
hdf.groupby(by="synapse_type").apply(lambda df: pandas.DataFrame(
    data={
        "total count": df.sum(), 
        "mean per cell": df.mean(), 
        "25%": df.quantile(q=0.25),
        "50%": df.quantile(q=0.5),
        "75%": df.quantile(q=0.75),
    })).droplevel(1)

,total count,mean per cell,25%,50%,75%
synapse_type,,,,,
autapse,14,1.166667,1.00,1.0,1.00
post,920639,2397.497396,1552.75,2368.0,3171.25
pre,15028,40.506739,3.00,6.0,48.50
